In [1]:
from collections import Counter

Warmup statistics for most common words

In [26]:
def read_file(filepath: str):
    words = []
    with open(filepath, "r") as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]  # strip whitespace and newline character
        for line in lines:
            words.extend(line.split(" "))
    return words


In [27]:
# german text
data_de_path = "./data/europarl-v7.de-en.lc.de"
de_words = read_file(data_de_path)

# english text
data_en_path = "./data/europarl-v7.de-en.lc.en"
en_words = read_file(data_en_path)

In [28]:
de_cnt = Counter(de_words)
en_cnt = Counter(en_words)

n = 10
de_most_cmn = de_cnt.most_common(n)
en_most_cmn = en_cnt.most_common(n)

print(f"Most common {n} words in german text:")
for word, cnt in de_most_cmn:
    print(f"Word: '{word}'; Count: {cnt}")

print(f"Most common {n} words in english text:")
for word, cnt in en_most_cmn:
    print(f"Word: '{word}'; Count: {cnt}")


Most common 10 words in german text:
Word: ','; Count: 18549
Word: 'die'; Count: 10521
Word: '.'; Count: 9733
Word: 'der'; Count: 9374
Word: 'und'; Count: 7028
Word: 'in'; Count: 4175
Word: 'zu'; Count: 3168
Word: 'den'; Count: 2976
Word: 'wir'; Count: 2863
Word: 'daß'; Count: 2738
Most common 10 words in english text:
Word: 'the'; Count: 19847
Word: ','; Count: 14320
Word: '.'; Count: 9920
Word: 'of'; Count: 9597
Word: 'to'; Count: 9059
Word: 'and'; Count: 7303
Word: 'in'; Count: 6237
Word: 'is'; Count: 4478
Word: 'that'; Count: 4441
Word: 'a'; Count: 4435


In [37]:
p_zebra = en_cnt["zebra"] / len(en_words)
print(f"Probability for 'zebra': {p_zebra}")

p_speaker = en_cnt["speaker"] / len(en_words)
print(f"Probability for 'speaker': {p_speaker}")

Probability for 'zebra': 0.0
Probability for 'speaker': 3.78883e-05


Language modeling

Translation modeling

Decoding